In [1]:
import pandas as pd
import myslack
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import pickle
import time
import re
from tqdm import tqdm
from tqdm import trange

import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [3]:
food_dict = {
    '프랜차이즈':3, '치킨':4, '피자':5, '양식':5, '중국집':6,
    '한식':7, '일식':8, '돈가스':8, '족발':9, '보쌈':9, 
    '야식':10, '분식':11, '카페':12, '디저트':12 }

In [23]:
int(driver.find_element_by_css_selector('#restaurant_count').text)

77

In [29]:
stretch_list_page()

  0%|          | 0/3 [00:00<?, ?it/s]

모든 음식점 리스트 불러오기 시작...


100%|██████████| 3/3 [00:09<00:00,  3.01s/it]

모든 음식점 리스트 불러오기 완료!


In [31]:
len(get_restaurant_list())

76

In [37]:
set_location('방이2동')

방이2동으로 위치 설정 하는중...
방이2동으로 위치 설정 완료!


In [38]:
go_to_category('치킨')

치킨 카테고리 페이지 로드중...
치킨 카테고리 페이지 로드 완료!


In [41]:
stretch_list_page()

  0%|          | 0/3 [00:00<?, ?it/s]

모든 음식점 리스트 불러오기 시작...


100%|██████████| 3/3 [00:09<00:00,  3.01s/it]

모든 음식점 리스트 불러오기 완료!


In [44]:
restaurant_list = get_restaurant_list()

In [45]:
search_restaurant(restaurant_list[0])

In [46]:
go_to_restaurant()

In [47]:
go_to_review()

리뷰 페이지로 로드중...
리뷰 페이지로 로드 완료!


In [ ]:
get_all_review_elements()

In [49]:
stretch_review_page()

  0%|          | 0/49 [00:00<?, ?it/s]

모든 리뷰 불러오기 시작...


100%|██████████| 49/49 [06:02<00:00,  7.41s/it]

모든 리뷰 불러오기 완료!


In [56]:
# 0. 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 1. 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(1)
    print(location+'으로 위치 설정 완료!')
    
# 2. 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(1)
    print(category+' 카테고리 페이지 로드 완료!')    

# 3. 카테고리(음식점 리스트) 페이지 모두 펼치기
def stretch_list_page():
    restaurant_count = int(driver.find_element_by_css_selector('#restaurant_count').text)
    scroll_count = int((restaurant_count/20))
    print('모든 음식점 리스트 불러오기 시작...')
    for _ in trange(scroll_count):
        try:
            scroll_bottom()
            time.sleep(2.5)
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 음식점 리스트 불러오기 완료!')
    
# 4. 해당 카테고리 음식점 리스트 리턴
def get_restaurant_list():
    restaurant_list=[]
    restaurants = driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
    return restaurant_list

# 5. 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[13]/form/div/input').send_keys(restaurant_name)    
    driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    time.sleep(2)

# 6. 검색한 음식점 클릭
def go_to_restaurant():
    driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[2]/div').click()
    time.sleep(1)

# 7. 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지로 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(1)
    print('리뷰 페이지로 로드 완료!')
    
# 8. 리뷰 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)
    
# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 10. 해당 음식점의 모든 리뷰 객체 리턴
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
        
# 11. 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(3)
    print('페이지 돌아가기 완료!')

In [57]:
def yogiyo_crawling(location, category):
    set_location(location) 
    go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감

    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                               'Total','Taste','Quantity','Delivery','Date'])

    print('Start {} - {} Crawling...'.format(location, category))
    try:
        stretch_list_page() # 카테고리(음식점 리스트) 페이지 모두 펼치기
        restaurant_list = get_restaurant_list() # 해당 카테고리 음식점 리스트 받아오기
        
        for restaurant_name in restaurant_list:
            search_restaurant(restaurant_name) # 음식점을 순서대로 검색
            go_to_restaurant() # 검색한 음식점 클릭             
            go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
            stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

            for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                try:
                    df.loc[len(df)] = { 
                        'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                        'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                        'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                        'Review':review.find_element_by_css_selector('p').text,
                        'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                        'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                        'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                        'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                        'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                    }
                except Exception as e:
                    print('리뷰 페이지 에러')
                    print(e)
                    pass

            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            time.sleep(2)
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
        
    print('End of {} {} Crawling!'.format(location, category))
    myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    return df

In [ ]:
%time chinese_df = yogiyo_crawling('방이2동','치킨')

방이2동으로 위치 설정 하는중...
방이2동으로 위치 설정 완료!
치킨 카테고리 페이지 로드중...


  0%|          | 0/3 [00:00<?, ?it/s]

치킨 카테고리 페이지 로드 완료!
Start 방이2동 - 치킨 Crawling...
모든 음식점 리스트 불러오기 시작...


100%|██████████| 3/3 [00:07<00:00,  2.51s/it]


모든 음식점 리스트 불러오기 완료!
리뷰 페이지로 로드중...


...

100%|██████████| 377/377 [01:39<00:00,  3.79it/s]


음식점 리스트 페이지로 돌아가는중...
페이지 돌아가기중...
페이지 돌아가기 완료!
리뷰 페이지로 로드중...


  0%|          | 0/20 [00:00<?, ?it/s]

리뷰 페이지로 로드 완료!
모든 리뷰 불러오기 시작...


 45%|████▌     | 9/20 [01:03<01:17,  7.07s/it]

In [ ]:
chicken_df.tail()

In [208]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기

WebDriverException: Message: chrome not reachable
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)


---